In [37]:
import pandas as pd
import regex as re

def remove_spaces(string):
    if string == '':
        return ''
    elif string == ' ':
        return ''
    if string[0] == ' ':
        string = string[1:]
    if string[-1] == ' ':
        string = string[:-1]
    return string

def remove_prefix(name, prefix):
    if name.startswith(prefix):
        name = name[len(prefix):].strip()
    return name

def process_name(name, abbreviations):
    for abbreviation in abbreviations:
        name = remove_prefix(name, abbreviation)
    return name.strip()

if __name__ == '__main__':
    char = {'id': [], 'name': []}
    df = pd.read_csv('/kaggle/input/thaijo1/test.csv')
    df = df.fillna('')

    abbreviations_df = pd.read_csv('/kaggle/input/thai-abbreviation/thai-abbreviation.csv')
    abbreviations = abbreviations_df['อักษรย่อ'].tolist()

    for idx, row in df.iterrows():
        names = []
        char['id'].extend([f"{row['_id']}_{i+1}" for i in range(10)])
        author = row['_source.author']
        if author != '':
            author_names = re.findall(r'\((.*?)\)', author)
            author = re.sub(r'\(.*?\)', '', author).strip()
            if author:
                names.append(author)
            names.extend(author_names)

        co_author = row['_source.co-author']
        if co_author != '':
            ca = co_author.split(',')
            for c in ca:
                c = c.strip()
                co_author_names = re.findall(r'\((.*?)\)', c)
                c = re.sub(r'\(.*?\)', '', c).strip()
                if c and c not in names:
                    names.append(c)
                names.extend(co_author_names)

        if len(names) == 0:
            char['name'].extend([None] * 10)
            continue

        names = [remove_spaces(name) for name in names]
        names = [re.sub(r'^[-*.\s]+|[-*.\s]+$', '', name) for name in names]
        names = [' '.join([n.capitalize() for n in name.split()]) for name in names]
        names = [process_name(name, abbreviations) for name in names]

        if len(names) > 10:
            names = names[:10]
        elif len(names) < 10:
            names.extend([''] * (10 - len(names)))

        char['name'].extend(names)

    char = pd.DataFrame(char)
    char.to_csv('/kaggle/working/predict.csv', index=False, encoding='utf-8')